In [2]:
from drawpile import drawPile
import numpy as np
import ILP
from hand import hand
'''
gather all of the tiles on the board and in your hand
count them up 0 to 2 for each tile, recorded in y array
y[i] array:
y[0:13] corresponds to Red tiles
y[13:26] corresponds to Blue tiles
y[26:39] corresponds to Yellow tiles
y[39:52] corresponds to Black tiles
y[52] corresponds to Joker

Ex: with hand R2 J0 Y8 K3 Y8 K6
y[1] = 1. y[33] = 2, y[41] = 1, y[44] = 1, y[52] = 1
all other y indices are 0

x[j] array:
as long as the total number of every possible group in the game
at each index is a 0,1, or 2
indicates how many times that group can be played based on hand+board

s[i][j] matrix:
each location i,j has 1 if tile i is in set j
0 if tile i not in set j

r[i] array:
tiles on your rack, same format as y[i]
y[i] <= r[i]
y[i] is tiles from rack that can be played, so if y[i] == r[i], you win

t[i] array:
tiles on the table, same format as y[i]

The hard part:
Maximize Sum of y[i] over all i∈{1,2...,53}
subject to: Sum of s[i][j]x[j] == t[i] + y[i]
            y[i] <= r[i]
            x[j] ∈ {0,1,2}
            y[j] ∈ {0,1,2}

source: doi:10.1093/comjnl/bxl033
'''
tileToIndex = []
for i in ['R','B','Y','K']:
    for j in range(13):
        tileToIndex.append(i+str(j+1))
tileToIndex.append('J0')
tileToIndex = tuple(tileToIndex)

exampleDraw = drawPile()
exampleHand = hand(exampleDraw,1)

exHandStrings = [x.string for x in exampleHand.hand]

exhaustiveList = ILP.generateSet(13,['R','B','K','Y'])
xarray = np.zeros(len(exhaustiveList))
for item in exhaustiveList:
    jokerCount = 0
    tileCounter = 0
    for el in item.group:
        if el.string in exHandStrings:
            tileCounter += 1
            if el.string == 'J0':
                exHandStrings.remove('J0')
                jokerCount += 1
    if tileCounter == len(item.group):
        xarray[exhaustiveList.index(item)] += 1
    exHandStrings.extend(jokerCount*['J0'])

In [3]:
print(exHandStrings)
print(xarray)
# print(exhaustiveList)
np.where(xarray>=1)

['K8', 'K3', 'K5', 'K13', 'Y9', 'R1', 'R2', 'K7', 'Y11', 'K2', 'B7', 'K6', 'J0', 'J0']
[0. 0. 0. ... 0. 0. 0.]


(array([  69,   72,   73,  187,  421,  426,  432,  436,  443,  447,  452,
         455,  456,  457,  458,  463,  467,  468,  469,  470,  471,  472,
         473,  474,  481,  485,  490,  622,  674,  681,  753,  764,  866,
         867,  870,  876, 1098, 1118, 1138, 1231, 1232, 1235, 1236, 1239,
        1243, 1245, 1247, 1249, 1251, 1254, 1262, 1263, 1265, 1268, 1269,
        1272, 1275, 1278, 1279, 1281, 1282, 1283, 1284, 1285, 1286, 1290,
        1294, 1295, 1296, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307,
        1308, 1309, 1310, 1311, 1312, 1320, 1321, 1324, 1326, 1327, 1331,
        1334, 1336, 1337, 1340, 1346, 1356, 1395, 1502, 1522, 1540, 1542,
        1544, 1560, 1566, 1580, 1607, 1613, 1641, 1679]),)

In [99]:
exhaustiveList = np.array(exhaustiveList)
groupsFromHand = np.where(xarray==1)
makeableGroups = exhaustiveList[groupsFromHand]
# for item in makeableGroups:
#     print(*[x.string for x in item.group])

In [46]:
from board import board
from group import group
b = board()
b.addGroups([exhaustiveList[20],exhaustiveList[237]])

In [53]:
list(np.array([1,2,3]))

[1, 2, 3]

In [47]:
b.board

[<group.group at 0x7f7df081f6a0>, <group.group at 0x7f7df085b9d0>]

In [100]:
'''
start with 1 group, remove tiles from hand/board combo
'''
### Need to fix for the joker at the end case
bestLen = -1
bestChoice = []
# makeableGroups = b.board + list(makeableGroups)

for i in range(len(makeableGroups)):
    handCopy = exampleHand.hand[:]
    handCopyStrings = [x.string for x in handCopy]
    boardCopy = []
    for item in b.board:
        boardCopy.extend(item.group)
    boardCopyStrings = [x.string for x in boardCopy]
    
    masterList = boardCopy + handCopy
    masterStrings = boardCopyStrings + handCopyStrings

    useThese = []
    for item in makeableGroups[i:]:
        tempGroup = []
        for el in item.group:
            try:
                masterList.pop(masterStrings.index(el.string))
                masterStrings.remove(el.string)
            except:
                masterList.extend(tempGroup)
                masterStrings.extend([x.string for x in tempGroup])
                tempGroup = []
                break
            tempGroup.append(el)

        if tempGroup != []:
            useThese.append(tempGroup)
    
    newsThese = []
    for i in useThese:
        newsThese.extend(i)
    useTheseStrings = [x.string for x in newsThese]
    boardCheck = True
    for el in boardCopyStrings:
        if el in useTheseStrings:
            useTheseStrings.remove(el)
        else:
            boardCheck = False
            break

    if boardCheck and bestLen < len(np.array(bestChoice, dtype=object).flatten()):
        bestChoice = useThese[:]
        bestLen = len(bestChoice)
        bestPlay = masterStrings[:]
             

for item in bestChoice:
    print([x.string for x in item])

print('\n')
print(bestPlay)
# print(handCopyStrings)
# print(exHandStrings)

['R6', 'R7', 'R8', 'R9', 'R10']
['R5', 'R6', 'R7', 'J0']
['K7', 'K8', 'K9']
['B4', 'B5', 'B6']
['K5', 'K6', 'K7']
['R1', 'R2', 'J0']
['J0', 'K2', 'K3']


['K13', 'B7', 'K8', 'Y9', 'Y11']


In [98]:
from tile import tile
b.addGroups([group([tile(4,'B'),tile(5,'B'),tile(6,'B')])])

In [66]:
print(*[x.string for x in b.board[1].group])

R5 R6 R7 J0
